# AWS IoT Connectivity and Security Basics


Recover variables from previous notebooks:

In [ ]:
%store -r home
print(home)
%store -r unique
print(unique)

To start the workshop we are going to create a Thing inside AWS IoT. A Thing should represent any phisycal thing in world: button, a raspberry pi, a lamp or even your own laptop can be a thing. All a thing need is a name, let's use that unique identifier for that.

In [ ]:
thing_name = "bp_thing_{}".format(unique)
%store thing_name
thing_name

In [ ]:
! aws iot create-thing --thing-name {thing_name}

# Mutual Authentication


Create a certificate and keys to authenticate your thing. 

In [ ]:
! mkdir -p "{home}/aws_iot"

certificate_file = "{}/aws_iot/certificate{}.pem".format(home,unique)
public_key = "{}/aws_iot/public-key{}.pem".format(home,unique)
private_key = "{}/aws_iot/private-key{}.pem".format(home,unique)

%store certificate_file
%store public_key
%store private_key

In [ ]:
certificateArn = ! aws iot create-keys-and-certificate \
  --set-as-active \
  --certificate-pem-outfile {certificate_file} \
  --public-key-outfile {public_key} \
  --private-key-outfile {private_key} \
  --query certificateArn \
  --output text
certificate_arn = certificateArn.s
certificate_arn

Download the root Certification Authority used by AWS IoT


In [ ]:
ca_file = "{}/aws_iot/ca{}.pem".format(home,unique)
%store ca_file
! wget https://www.symantec.com/content/en/us/enterprise/verisign/roots/VeriSign-Class%203-Public-Primary-Certification-Authority-G5.pem -O {ca_file}

Double check that you have the 4 files required for TLS mutual authentication used by AWS IoT
1. Private Key
1. Public Key
1. Certificate
1. Root CA

In [ ]:
! printf "Private Key {private_key} " && [ -f {private_key} ] && printf "found" || printf "not found"
! printf "\nPublic Key {public_key} " && [ -f {public_key} ] && printf "found" || printf "not found"
! printf "\nCertificate {certificate_file} " && [ -f {certificate_file} ] && printf "found" || printf "not found"
! printf "\nCertification Authority {ca_file} " && [ -f {ca_file} ] && printf "found" || printf "not found"

# Authorization

The thing now can be properly identified, but not yet allowed to do anything. As usual in AWS, permissions are denied by default unless explicitly allowed through and IAM policy. The following policy allows all API invocations on AWS IoT targeting all resources.

In [ ]:
#TODO: Use a more restrictive security policy
policy_document = '''{
  "Version": "2012-10-17",
  "Statement": [{
    "Effect": "Allow",
    "Action": "iot:*",
    "Resource": "*"}]
}'''

In [ ]:
policy_name = "bp_policy_{}".format(unique)
policyArn = ! aws iot create-policy \
    --policy-name {policy_name} \
    --policy-document '{policy_document}' \
    --query policyArn \
    --output text
policyArn.s

The security policy is attached to the thing certificate, which in turn is attached to the thing. 

In [ ]:
! aws iot attach-policy \
    --policy-name {policy_name} \
    --target {certificate_arn}

In [ ]:
! aws iot attach-thing-principal \
  --thing-name {thing_name} \
  --principal {certificate_arn}

# Connecting to AWS IoT

AWS IoT provides account-specific endpoints for your things to connect:

In [ ]:
endpoint_address = !aws iot describe-endpoint --query endpointAddress --output=text
endpoint_address = endpoint_address.s
%store endpoint_address
endpoint_address

With the endpoint and credentials we can create an MQTT client instance:

In [ ]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient

client_id="bp_client_{}".format(unique)
mqtt = AWSIoTMQTTClient(client_id)
mqtt.configureEndpoint(endpoint_address, 8883)
mqtt.configureCredentials(ca_file, private_key, certificate_file)
mqtt.configureConnectDisconnectTimeout(600)
mqtt

# Receiving Messages

Connect to the service to receive messages:

In [ ]:
mqtt.connect()

MQTT topics are hierarchical message queues. The ```subscribe``` method will listen for messages in the topic with the specified quality of service and invoke the callback for each message. 

In [ ]:
topic_name ="bp_topic/data"
%store topic_name

subscribe_qos = 1 #at-least-once delivery

def custom_callback(client, userdata, message):
    msg = message.payload.decode("utf-8")
    print("\nSuper phone received a message: \n {}".format(msg))
    
mqtt.subscribe(topic_name, subscribe_qos, custom_callback)

Try sending messages to this topic using the [AWS Core IoT Console](https://console.aws.amazon.com/iot/home?region=us-east-1#/test)

# Publishing Messages

Publishing messages to a topic is just as simple. Let's create a message structure for our blood pressure measurements:

In [ ]:
data = {"id": "fa91d2de-24a1-4052-8b58-4f2ab743f20e",
        "name": "Clara Oswald", 
        "systolic": 120, 
        "diastolic": 80, 
        "time_stamp": 1541059348245}

import json
payload = json.dumps(data, indent=4)
%store payload

In [ ]:
publish_qos = 0 # fire and forget
mqtt.publish(topic_name, payload , publish_qos)

In [ ]:
#Wait for it
import time
time.sleep(1)

The subscribe_qos and publish_qos arguments refer to the quality of service parameter of the MQTT protocol. Selecting the QoS level is a tradeoff between delivery guarantee and performance, see the following resources for more infomation:

https://www.hivemq.com/blog/mqtt-essentials-part-6-mqtt-quality-of-service-levels

https://docs.aws.amazon.com//iot/latest/developerguide/protocols.html

All working fine, done for now, disconnect.

In [ ]:
mqtt.disconnect()

Although sending and receiving messages is quite simple, device state management can be complex. For example, consider an IoT lightbulb controled by a mobile app, sending on/off requests and displaying the current state. If any of the mobiles or the lightbulb is temporarilly disconnected, messages will be lost, requiring state reconciliation to display  the light correctly. For more sophisticated devices, state merging and management can consume significant development efforts.

AWS IoT Core can store and manage the "device shadow" on the cloud and calculate the difference between the reported and desired device state. Let's see how that works in the [Managing Device State](aws-iot-shadow.ipynb) notebook.